In [1]:
###READ IN THE LOG FILE
import csv
import cv2
import numpy as np
import sklearn
lines=[]
imagePaths=[]
measurements=[]
with open('/Volumes/USBDRIVE/Udacity/Term1/CarND-Behavioral-Cloning-P3/recording10/driving_log.csv') as csvfile:
#with open('/Volumes/USBDRIVE/Udacity/Term1/CarND-Behavioral-Cloning-P3/recordings/driving_log.csv') as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        #Append center camera image
        imagePaths.append(line[0])
        #Append left camera image
        imagePaths.append(line[1])
        #Append right camera image
        imagePaths.append(line[2])
        #Append center camera measurement
        measurements.append(float(line[3]))
        #Append left camera measurement, corrected by a factor
        measurements.append(float(line[3])+0.2)
        #Append right camera measurement, corrected by a factor
        measurements.append(float(line[3])-0.2)
        

In [2]:
###SETUP GENERATOR FUNCTION TO STREAM DATA INSTEAD OF PRE-LOADING INTO MEMORY
def generator(samples, batch_size=128):

    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        ###Shuffle the samples so as to remove any sequential data bias
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for imagePath, measurement in batch_samples:
                ###READ IN IMAGES, CONVERT TO RGB SPACE
                originalImage = cv2.imread(imagePath)
                image = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
                images.append(image)
                angles.append(measurement)
                # Augment the original image by Fliping it
                images.append(cv2.flip(image,1))
                # Since image is flipped, flip the steering direction output as well
                angles.append(measurement*-1.0)

            
            inputs = np.array(images)
            outputs = np.array(angles)
            ###Yield data via generator
            yield sklearn.utils.shuffle(inputs, outputs)

In [3]:
# Splitting samples into training & validation, use 20% of data for validation
from sklearn.model_selection import train_test_split
samples = list(zip(imagePaths, measurements))
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

print('Train samples: {}'.format(len(train_samples)))
print('Validation samples: {}'.format(len(validation_samples)))


Train samples: 7996
Validation samples: 2000


In [4]:
#setup generators, feed data in batches of 32 images to conserve memory
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [5]:
#DEFINE THE MODEL
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU,Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.advanced_activations import LeakyReLU

###INITIALIZE Keras sequential model
model = Sequential()
###Crop the images to remove extraneous information, focus on the immediate road ahead
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(160,320,3)))###Pre-process the data, center the data
model.add(Lambda(lambda x:x/127.5-1,input_shape=(160,320,3) ))

###USE THE MODEL DEFINED IN COMMA.AI Steering model
#https://github.com/commaai/research/blob/master/train_steering_model.py
model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
model.add(LeakyReLU(alpha=.001))   # add an advanced activation
#model.add(ELU())
model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(LeakyReLU(alpha=.001))   # add an advanced activation
#model.add(ELU())
model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(Flatten())
model.add(Dropout(.2))
model.add(LeakyReLU(alpha=.001))   # add an advanced activation
#model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(LeakyReLU(alpha=.001))   # add an advanced activation
#model.add(ELU())
model.add(Dense(1))
###END COMMA.AI MODEL
###COMPILE USING ADAM OPTIMIZER, SO THAT LEARNING RATE DOESNT HAVE TO BE SET MANUALLY
model.compile(optimizer="adam", loss="mse")



Using TensorFlow backend.


In [6]:
###FEED THE GENERATOR DATA TO THE MODEL, TRAIN
### 3 epochs was sufficient, beyond that, seemed to overfit
history_object = model.fit_generator(train_generator, samples_per_epoch= \
                 len(train_samples), validation_data=validation_generator, \
                 nb_val_samples=len(validation_samples), nb_epoch=3, verbose=1)
###save the trained model
model.save('model.h5')
print(history_object.history.keys())
print('Loss')
print(history_object.history['loss'])
print('Validation Loss')
print(history_object.history['val_loss'])

Epoch 1/3
7936/7996 [============================>.] - ETA: 0s - loss: 0.0450

/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


8000/7996 [==============================] - 101s - loss: 0.0450 - val_loss: 0.0356
Epoch 2/3
8056/7996 [==============================] - 101s - loss: 0.0293 - val_loss: 0.0276
Epoch 3/3
8000/7996 [==============================] - 100s - loss: 0.0281 - val_loss: 0.0259
dict_keys(['val_loss', 'loss'])
Loss
[0.044971554368734359, 0.029347745666526398, 0.028115500599145891]
Validation Loss
[0.035609549609944224, 0.027640409649364532, 0.025910924363415688]


In [ ]:
import csv
import cv2
import numpy as np
import sklearn
imagePath='flip.jpg'
originalImage = cv2.imread(imagePath)
#image = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
newImage=cv2.flip(originalImage,1)
cv2.imwrite('flipped.jpg',newImage)


In [ ]:
from keras.utils.visualize_util import plot  
plot(model, to_file='model.png')